In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from gtts import gTTS
import matplotlib
import numpy 
from traitlets.config.manager import BaseJSONConfigManager
path = "/Users/scot/anaconda3/envs/py36/etc/jupyter/nbconfig"
cm = BaseJSONConfigManager(config_dir=path)
cm.update("livereveal", {"autolaunch": True,
                         "theme": "sky",
                        } 
)



#Supress default INFO logging
# The UT Dallas Art Science Lab Training module 
print ('The UTDallas ArtSci Lab Sonification "Half-Empty/Half-Full" Testing Module.')
print("\n")
print("Refresher instructions for those unfamiliar with these sonification training modules" )
print("\n Basic Instructions for each cell :")
print('1. Press "Shift + Enter" at every new cell to initiate display/execute that cell')
print("2. Hit the Space Bar to go to next section (cell)")
print("\n")
print("These are the two main actions that you need to move forward, cell by cell in each unit")
print("This is how this system functions")
print("Do these now")

The UTDallas ArtSci Lab Sonification "Half-Empty/Half-Full" Testing Module.


Refresher instructions for those unfamiliar with these sonification training modules

 Basic Instructions for each cell :
1. Press "Shift + Enter" at every new cell to initiate display/execute that cell
2. Hit the Space Bar to go to next section (cell)


These are the two main actions that you need to move forward, cell by cell in each unit
This is how this system functions
Do these now


## As Before, Please put on your best headphones on and Set your system Volume to 50%.
# Now a little game with this simple sonificaiton, Can you get 80% or higher? ... !
## Use Space Bar or arrows at bottom right to navigate to next section

In [3]:
import random
import time
from IPython.display import Image, display, clear_output
from ipywidgets import Button, HBox, VBox,Layout
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from gtts import gTTS
import os
import numpy
import ctcsound
import platform
import os
speechflag = 0
if (platform.system()=='Windows'):
    speechflag = 2
if (platform.system()!='Windows'):
    speechflag = 1

print ("Listen to the Sonification, Set the Slider to what Percentage you think it represents and Submit Response. You will undergo 5 trials")
count=0
accuracy =0

pan = 0
user_input=0
cs = ctcsound.Csound()
index = 0

csd = '''
<CsoundSynthesizer>
<CsOptions>
-o dac
</CsOptions>
<CsInstruments>
;
sr = 44100
ksmps = 32
nchnls = 2
0dbfs = 1

giSine    ftgen     0, 0, 2^10, 10, 1

instr 1 ;master instrument
inumparts =         p4 ;number of partials
ibasfreq  =         200 ;base frequency
ipart     =         1 ;count variable for loop
;loop for inumparts over the ipart variable
;and trigger inumpartss instanes of the subinstrument
loop:
ifreq     =         ibasfreq * ipart
iamp      =         1/ipart/inumparts
          event_i   "i", 10, 0, p3, ifreq, iamp
          loop_le   ipart, 1, inumparts, loop
endin

instr 10 ;subinstrument for playing one partial
ifreq     =         p4 ;frequency of this partial
iamp      =         p5 ;amplitude of this partial
aenv      transeg   0, .01, 0, iamp, p3-0.1, -10, 0
apart     poscil    aenv, ifreq, giSine
          outs      apart, apart
endin



</CsInstruments>
<CsScore>
f 0 14400
f 1 0 1024 10 1 

</CsScore>

</CsoundSynthesizer>
'''
try:
    pt
except NameError:
    var_exists = False
else:
    pt.stop()
    pt.join()
    time.sleep(2)
    
cs.compileCsdText(csd)
cs.start()
pt = ctcsound.CsoundPerformanceThread(cs.csound())
pt.play()



def f(percentage):
    global user_input
    user_input = percentage



def redraw(): 
    
    global index
    global accuracy
    sonibutton = widgets.Button(description = 'Listen to Sonification')
    answerbutton = widgets.Button(description='Submit Response')
    choices = random.sample(range(100), 4)
    choices = list(map(str, choices))
    correct = random.choice(choices)
    index = int(correct)

    #display(Image(correct))
    #display(correct)
    time.sleep(0.5)
    
    
    #display(button)
    #button.on_click(on_button_clicked)
    
    #buttons = [widgets.Button(description = choice) for choice in choices]
    #sonibutton = [widgets.Button(description = 'Listen to Sonification')]
    interact (f, percentage=(0,100,1))
    #answerbutton = [widgets.Button(description='Submit Input')]
    
    #container = widgets.HBox(children=buttons)

    
    left_box = VBox([(sonibutton)])
    right_box = VBox([(answerbutton)])
    #HBox([left_box, right_box])
    container = widgets.HBox([left_box,right_box])
    print("Trial " + str(count+1))
    display(container)
    


    def ans_button_clicked(b):
            global count
            global accuracy
            count = count + 1
            tts = gTTS(text='Input Submitted', lang='en')
            tts.save("answer.mp3") 
            
            if (speechflag==1):
                os.system("afplay answer.mp3")
            if (speechflag==2):
                os.system("cmdmp3 answer.mp3")
            
            text=list()
            text.append(index)
            text.append(user_input)
            text.append(index-user_input)
            accuracy = accuracy + abs(index -user_input)
            with open('drop_responses.csv','a') as file:
                file.write('\n')
                for line in text:
                    file.write(str(line))
                    file.write(',')
            time.sleep(2)
            container.close()
            clear_output()
            if count <5:
                redraw()
            if count == 5:
                msg = widgets.Button(description = 'Thank you for finishing this module',layout=Layout(width='50%', height='80px'))
                display(msg)
                print("Your accuracy of response is " + str(100-(accuracy/5)) + "%")
                pt.stop()
                pt.join()
                file.close()
            
        
    def son_button_clicked(b):
            
            #tts = gTTS(text='Playing Sonification', lang='en')
            #tts.save("answer.mp3") 
            #os.system("afplay answer.mp3")
            
            in_min  = 0
            in_max = 100
            out_min=0
            out_max = 20
        
            global index
    
            freq = (index - in_min) * (out_max - out_min) / (in_max - in_min) + out_min  
            #print(freq)  
    
            pt.scoreEvent(False, 'i', (1, 0, 3,freq))
            time.sleep(4.5)
             
            
        
        
    answerbutton.on_click(ans_button_clicked)
    sonibutton.on_click(son_button_clicked)


 
   
    

redraw()





Listen to the Sonification, Set the Slider to what Percentage you think it represents and Submit Response. You will undergo 5 trials


interactive(children=(IntSlider(value=50, description='percentage'), Output()), _dom_classes=('widget-interact…

Trial 1


In [ ]:
##  You have completed this set of Training Exploring and Testing!!!

Now that you have been evaluated on this straight forward sound representation,
it is time to move on to the next phase of seeing what other sonifications are available. 
Go back to the list of available notebooks and select the next set of sonification to check out. 
